In [1]:
import numpy as np
import pandas as pd
import csv
import geopy
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

## Indetify missing geo coord import ones and create the list of addresses for re-trial

In [2]:
df = pd.read_csv('data/first_round_geo_imported.csv')

In [52]:
len(df)

5229

In [3]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,...,short_inspect_date,short_violation_id,inspect_date,p1_3,p4_6,p7_9,p10_12,p13_18,p19_24,p25_36
0,0,0,0,1757,Dar Bar Pakistani/Indian Cusine,1412 Polk St,San Francisco,CA,94109,37.789784,...,20170928,103131,2017-09-28,0,0,5,0,0,6,5
1,1,1,1,4864,DRAGON CITY BAKERY & CAFE,2367 MISSION St,San Francisco,CA,94110,37.759174,...,20161206,103157,2016-12-06,0,6,0,0,5,5,3
2,2,2,2,79782,Deli 23,2449 23rd St,San Francisco,CA,94110,37.754343,...,20160503,103120,2016-05-03,4,0,2,0,3,3,2
3,3,3,3,73840,L'acajou Bakery and Cafe,498 09th St Ste. C,San Francisco,CA,94103,NaN,...,20171207,103105,2017-12-07,0,5,0,0,6,0,0
4,4,4,4,76437,Sweetheart Cafe,909 Grant Ave,San Francisco,CA,94108,37.795305,...,20160329,103113,2016-03-29,0,11,0,0,0,6,6


In [4]:
mask = df['business_latitude'].isnull().values

In [5]:
df[mask].head(2)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,...,short_inspect_date,short_violation_id,inspect_date,p1_3,p4_6,p7_9,p10_12,p13_18,p19_24,p25_36
3,3,3,3,73840,L'acajou Bakery and Cafe,498 09th St Ste. C,San Francisco,CA,94103,NaN,...,20171207,103105,2017-12-07,0,5,0,0,6,0,0
11,11,11,11,82880,Don Pablo,Off The Grid,San Francisco,CA,NaN,NaN,...,20160309,103148,2016-03-09,0,0,0,3,0,6,2


In [6]:
len(df[mask])

742

In [7]:
# 742 businesses where geo coords were not transferred
df_742 = df[mask]

In [8]:
address_no_geocoord = df_742['business_address']

In [9]:
address_nogeo_lst = address_no_geocoord.values

In [10]:
num_vars = ['Off the Grid', 'Off The Grid', 'OFF The Grid', 'OFF THE GRID', 'Approved private locations', 'Approved Private Locations','Approved Locations']
L_nogeo = []
for address in address_nogeo_lst:
    #if address != 'Off the Grid' and address != 'Off The Grid':
    if ~(address in num_vars):
        L_nogeo.append(address)

In [1]:
#L_nogeo

In [34]:
## Let's run geopy

In [11]:
geolocator = Nominatim(timeout = 20)

In [12]:
location = geolocator.geocode("2740 Mission St SF CA")
print(location.address, location.longitude, location.latitude)

Mission Street, St. Mary's Park, SF, California, 94134, United States of America -122.4291725 37.7308875


In [13]:
# add SF CA to addresses
L_nogeo_SFCA = []
for address in L_nogeo:
    L_nogeo_SFCA.append(address + ' SF CA')

In [14]:
def do_geocode2(address):
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        time.sleep(1)
        return do_geocode(address)

In [84]:
write_file = "data/geo_coords_sf_742.csv"
with open(write_file, "w") as output:
    for address in L_nogeo_SFCA[0:50]:  #742
        location = do_geocode2(address)
        if location != None:
            output.write(address + ',' + str(location.longitude) + ',' + str(location.latitude) +  '\n')

In [85]:
# show the list of restaurants whose geo coords are grabbed from geopy
with open('data/geo_coords_sf_742.csv', 'r') as f:
    reader = csv.reader(f)
    counter = 0
    for row in reader:
        counter += 1
        print(row)
        if counter > 20: break
print(counter)

['Off The Grid SF CA', '-122.4314265', '37.806651']
['2301 Market St SF CA', '-122.3987948', '37.7913412']
['2740 Mission St SF CA', '-122.4291725', '37.7308875']
['2348 Mission Street SF CA', '-122.4291725', '37.7308875']
['5937 Geary Blvd SF CA', '-122.4255764', '37.7851991']
['200 Columbus St SF CA', '-122.4088136', '37.7994755']
['977 A North Point St SF CA', '-122.423183959184', '37.8052633877551']
['1548 Ocean Ave SF CA', '-122.4514762', '37.7229098']
['2331 Chestnut St SF CA', '-122.441710636423', '37.79976935']
['2404 California St SF CA', '-122.3975149', '37.7935273']
['5601 Geary Blvd SF CA', '-122.4255764', '37.7851991']
['401 13th St SF CA', '-122.4176974', '37.7698728']
['2794 California St SF CA', '-122.3975149', '37.7935273']
['1919 Ocean Ave SF CA', '-122.4514762', '37.7229098']
['3985 17th St SF CA', '-122.409393', '37.764369']
['295 B Orizaba Ave SF CA', '-122.462700111111', '37.7141915555556']
['Off The Grid SF CA', '-122.4314265', '37.806651']
['3348 Steiner St SF C

## Import geo coords to the dataframe

In [86]:
df2 = df_742

In [87]:
df2 = df2.reset_index()

In [88]:
with open('data/geo_coords_sf_742.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        address = row[0]
        longitude = row[1]
        latitude = row[2]
        n = len(df2)
        for idx in range(n):
            df_address = df2.loc[idx,'business_address']
            if df_address == address[:-6]:  # To remove ' SF CA' at the end of addresses
                df2.loc[idx,'business_longitude'] = longitude
                df2.loc[idx,'business_latitude'] = latitude

In [2]:
#df2['business_latitude']

In [52]:
# count of no geo coords
mask = df['business_latitude'].isnull().values

In [53]:
mask.sum()

2174

In [27]:
df.count()

Unnamed: 0               5229
Unnamed: 0.1             5229
business_id              5229
business_name            5229
business_address         5229
business_city            5229
business_state           5229
business_postal_code     5070
business_latitude        2998
business_longitude       2998
business_location        2998
business_phone_number    1582
inspection_id            5229
inspection_date          5229
inspection_score         4808
inspection_type          5229
violation_id             5229
violation_description    5229
risk_category            5229
short_inspect_date       5229
short_violation_id       5229
inspect_date             5229
p1_3                     5229
p4_6                     5229
p7_9                     5229
p10_12                   5229
p13_18                   5229
p19_24                   5229
p25_36                   5229
dtype: int64

In [17]:
df.to_csv('data/first_round_geo_imported.csv')